# Data preprocessing of train and test set values

A preliminary lookup at the training and test set values reveal that there are many categories for some variables. Howwever, some of these categories refer to the same item but have different spelling. As such, data cleaning is necessary. 

Furthermore, the train and test set values differ in the item categories. Models may have a hard time classifying the test set if the test set categories are not found in the training set. One way to deal with this is to combine the training and test sets, pre-process the data at one shot, before splitting them back to training and test sets. This is further explained below.

## Load data

In [30]:
trainval <- read.csv('train-set-values.csv')
trainlab <- read.csv('train-set-labels.csv')
train <- merge(trainlab,trainval,by='id')
head(train)

id,status_group,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,non functional,0,2012-11-13,Tasaf,0,TASAF,33.12583,-5.118154,Mratibu,...,unknown,milky,milky,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
1,functional,0,2011-03-05,Shipo,1978,SHIPO,34.77072,-9.395642,none,...,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
2,functional,0,2011-03-27,Lvia,0,LVIA,36.11506,-6.279268,Bombani,...,per bucket,soft,good,insufficient,insufficient,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
3,functional,10,2013-06-03,Germany Republi,1639,CES,37.14743,-3.187555,Area 7 Namba 5,...,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
4,non functional,0,2011-03-22,Cmsr,0,CMSR,36.16489,-6.099289,Ezeleda,...,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,hand pump,hand pump
5,functional,50,2011-02-26,Private,28,Private,39.28612,-6.972403,Kwa Namaj,...,per bucket,soft,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe


In [31]:
test <- read.csv('test-set-values.csv')

In [2]:
summary(train)

       id                         status_group     amount_tsh      
 Min.   :    0   functional             :32259   Min.   :     0.0  
 1st Qu.:18520   functional needs repair: 4317   1st Qu.:     0.0  
 Median :37062   non functional         :22824   Median :     0.0  
 Mean   :37115                                   Mean   :   317.7  
 3rd Qu.:55657                                   3rd Qu.:    20.0  
 Max.   :74247                                   Max.   :350000.0  
                                                                   
    date_recorded                      funder        gps_height    
 2011-03-15:  572   Government Of Tanzania: 9084   Min.   : -90.0  
 2011-03-17:  558                         : 3635   1st Qu.:   0.0  
 2013-02-03:  546   Danida                : 3114   Median : 369.0  
 2011-03-14:  520   Hesawa                : 2202   Mean   : 668.3  
 2011-03-16:  513   Rwssp                 : 1374   3rd Qu.:1319.2  
 2011-03-18:  497   World Bank            : 1349

In [22]:
# get column names
names(trainval)

[1] "id"                    "amount_tsh"            "date_recorded"        
 [4] "funder"                "gps_height"            "installer"            
 [7] "longitude"             "latitude"              "wpt_name"             
[10] "num_private"           "basin"                 "subvillage"           
[13] "region"                "region_code"           "district_code"        
[16] "lga"                   "ward"                  "population"           
[19] "public_meeting"        "recorded_by"           "scheme_management"    
[22] "scheme_name"           "permit"                "construction_year"    
[25] "extraction_type"       "extraction_type_group" "extraction_type_class"
[28] "management"            "management_group"      "payment"              
[31] "payment_type"          "water_quality"         "quality_group"        
[34] "quantity"              "quantity_group"        "source"               
[37] "source_type"           "source_class"          "waterpoint_type"      
[40] "waterpoint_type_group"

## Determine number of unique id

We want to determine if a particular waterpoint has multiple data entries. If there are multiple data entries per waterpoint, the data is akin to a time-series data and has to be dealt with differently.

In [2]:
length(unique(train$id))

[1] 59400

Luckily, the number of entries correspond to the number of unique id. We do not have to deal with a time-series data.

## Select predictor variables

We see that some variables explain the same thing but at different levels of granularity. Here, we identify whether it is a one-to-one matching. If it is, we will take the more ganular data. However, if the variable has too many factor levels, we can take the superset instead. 

For a one-to-many matching, we have to include both variables in the model.

In [2]:
# library to create pivot tables
library(reshape)

Warning message:
"package 'reshape' was built under R version 3.6.3"

In [24]:
cast(trainval,extraction_type~extraction_type_class)
# extraction, extraction_type, extraction_type_class: extraction is the most granular

Using waterpoint_type_group as value column.  Use the value argument to cast to override this choice
Aggregation requires fun.aggregate: length used as default


extraction_type,gravity,handpump,motorpump,other,rope pump,submersible,wind-powered
afridev,0,1770,0,0,0,0,0
cemo,0,0,90,0,0,0,0
climax,0,0,32,0,0,0,0
gravity,26780,0,0,0,0,0,0
india mark ii,0,2400,0,0,0,0,0
india mark iii,0,98,0,0,0,0,0
ksb,0,0,0,0,0,1415,0
mono,0,0,2865,0,0,0,0
nira/tanira,0,8154,0,0,0,0,0
other,0,0,0,6430,0,0,0


In [25]:
cast(trainval,management~management_group)
# management, management_group: management is more granular

Using waterpoint_type_group as value column.  Use the value argument to cast to override this choice
Aggregation requires fun.aggregate: length used as default


management,commercial,other,parastatal,unknown,user-group
company,685,0,0,0,0
other,0,844,0,0,0
other - school,0,99,0,0,0
parastatal,0,0,1768,0,0
private operator,1971,0,0,0,0
trust,78,0,0,0,0
unknown,0,0,0,561,0
vwc,0,0,0,0,40507
water authority,904,0,0,0,0
water board,0,0,0,0,2933


In [14]:
cast(trainval,water_quality~quality_group)
# water_quality, quality_group: water_quality is more granular

Using waterpoint_type_group as value column.  Use the value argument to cast to override this choice
Aggregation requires fun.aggregate: length used as default


water_quality,colored,fluoride,good,milky,salty,unknown
coloured,490,0,0,0,0,0
fluoride,0,200,0,0,0,0
fluoride abandoned,0,17,0,0,0,0
milky,0,0,0,804,0,0
salty,0,0,0,0,4856,0
salty abandoned,0,0,0,0,339,0
soft,0,0,50818,0,0,0
unknown,0,0,0,0,0,1876


In [15]:
cast(trainval,quantity~quantity_group)
# quantity, quantity_group: equally granular

Using waterpoint_type_group as value column.  Use the value argument to cast to override this choice
Aggregation requires fun.aggregate: length used as default


quantity,dry,enough,insufficient,seasonal,unknown
dry,6246,0,0,0,0
enough,0,33186,0,0,0
insufficient,0,0,15129,0,0
seasonal,0,0,0,4050,0
unknown,0,0,0,0,789


In [17]:
cast(trainval,source~source_class)
# source, source_type, souce_class: source is the most granular

Using waterpoint_type_group as value column.  Use the value argument to cast to override this choice
Aggregation requires fun.aggregate: length used as default


source,groundwater,surface,unknown
dam,0,656,0
hand dtw,874,0,0
lake,0,765,0
machine dbh,11075,0,0
other,0,0,212
rainwater harvesting,0,2295,0
river,0,9612,0
shallow well,16824,0,0
spring,17021,0,0
unknown,0,0,66


In [4]:
cast(trainval,waterpoint_type~waterpoint_type_group, value='amount_tsh')
# waterpoint_type and waterpoint_type_group: waterpoint_type is more granular

Aggregation requires fun.aggregate: length used as default


waterpoint_type,cattle trough,communal standpipe,dam,hand pump,improved spring,other
cattle trough,116,0,0,0,0,0
communal standpipe,0,28522,0,0,0,0
communal standpipe multiple,0,6103,0,0,0,0
dam,0,0,7,0,0,0
hand pump,0,0,0,17488,0,0
improved spring,0,0,0,0,784,0
other,0,0,0,0,0,6380


## Select predictor variables

In [13]:
# a library for data manipulation
library(dplyr)

Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



### Combine DataFrame for feature engineering

Without combining the DataFrames, I faced the problem "type of predictors in new data do not match that of the training data" when I tried to predict the randomForest model on the test set. This is due to different factor levels in the training and test sets. 

To overcome this problem, we first combine the DataFrame for feature engineering. Since the test set does not have _status_group_, we use a dummy value. After selecting the factor levels, we split them back to the training and test sets.

In [32]:
test$status_group <- 'unknown'

In [33]:
combine <- rbind(train,test)

### Reduce factor levels for variables with many factor levels

In [34]:
length(unique(combine$installer))

[1] 2411

Understandably, there are many different installers. We reduce the number of factor levels for the installer variable by selecting the top 15 values and setting the rest as 'other'.

However, lets first observe the data.

In [35]:
combine %>% group_by(installer) %>% summarise(totals = n())
summary(combine$installer)

installer,totals
,4532
-,3
0,980
A.D.B,1
AAR,5
Aartisa,1
ABASIA,34
ABD,1
ABDALA,1
Abdallah Ally Wazir,1


DWE                               
                        21751                          4532 
                   Government                           RWE 
                         2282                          1498 
                        Commu                        DANIDA 
                         1347                          1305 
                         KKKT                        Hesawa 
                         1120                          1070 
                            0                          TCRS 
                          980                           887 
                          CES            Central government 
                          765                           764 
                        DANID                     Community 
                          690                           687 
                       HESAWA              District Council 
                          679                           663 
                 World vision                         TASAF 
                          517                           504 
                          LGA                        WEDECO 
                          501                           496 
             District council                         Gover 
                          490                           483 
                        AMREF                         TWESA 
                          398                           395 
                           WU                          Dmdd 
                          377                           356 
                         ACRA                  World Vision 
                          346                           326 
                         SEMA                            DW 
                          311                           308 
                        OXFAM                            Da 
                          288                           287 
                       UNICEF                          Gove 
                          285                           276 
                    Kiliwater                          FinW 
                          271                           267 
                    Villagers                         NORAD 
                          262                           260 
   Sengerema Water Department                 Idara ya maji 
                          258                           254 
                           DH                          DWSP 
                          246                           239 
         Lawatefuka water sup          Magadini-Makiwaru wa 
                          229                           226 
                       Distri                            RC 
                          224                           219 
                           FW          KKKT _ Konde and DWE 
                          214                           209 
          Handeni Trunk Main(                           WVT 
                          208                           206 
                          MWE                         Centr 
                          198                           195 
                        Norad                    Fini Water 
                          190                           188 
                           Is                        Kuwait 
                          187                           187 
                        JAICA                       Private 
                          184                           183 
                        SHIPO                         RWSSP 
                          183                           180 
                 Central govt                         GOVER 
                          175                           170 
                      Artisan                           ISF 
                          169                           168 
                      Mission                     RC CHURCH 
                          167                           162 


We realise that the same installer has multiple data entries because they are represented in different formats due to punctuation differences or spelling errors. Thus, we have to first use a common value for these installers.

In [36]:
# Reword values for values that were represented in different formats
change_var <- function(table){
    table$installer <- tolower(table$installer)
    table$installer[table$installer == 'central government'] <- 'government'
    table$installer[table$installer == 'cebtral government'] <- 'government'
    table$installer[table$installer == 'central govt'] <- 'government'
    table$installer[table$installer == 'centr'] <- 'government'
    table$installer[table$installer == 'gove'] <- 'government'
    table$installer[table$installer == 'gover'] <- 'government'
    table$installer[table$installer == 'distri'] <- 'district council'
    table$installer[table$installer == 'counc'] <- 'district council'
    table$installer[table$installer == 'council'] <- 'district council'
    table$installer[table$installer == 'district counci'] <- 'district council'
    table$installer[table$installer == 'district water department'] <- 'district council'
    table$installer[table$installer == 'commu'] <- 'community'
    table$installer[table$installer == 'commu'] <- 'community'
    table$installer[table$installer == 'adra /community'] <- 'adra'
    table$installer[table$installer == 'adra/ Community'] <- 'adra'
    table$installer[table$installer == 'adra/government'] <- 'adra'
    table$installer[table$installer == 'adra /government'] <- 'adra'
    table$installer[table$installer == 'adra/community'] <- 'adra'
    table$installer[table$installer == 'world vission'] <- 'world vision'
    table$installer[table$installer == 'would vission'] <- 'world vision' 
    table$installer[table$installer == 'world vision & community'] <- 'world vision'
    table$installer[table$installer == 'danid'] <- 'danida'
  return(table)
}

combine <- change_var(combine)

# note: this is a non-exhaustive list. I tried to identify as many values as possible

In [37]:
# Select the top 15 values and change the rest to "others"
combine$installer[combine$installer %in% c(" ", "", "0", "_", "-")] <- "other"
installer_top <- names(summary(as.factor(combine$installer)))[1:15]
combine$installer[!(combine$installer %in% installer_top)] <- "other"
table(combine$installer, combine$status_group)

                  
                   functional functional needs repair non functional unknown
  amref                   189                       6            248      95
  ces                     538                       1             71     155
  community              1113                      73            434     424
  danida                 1037                      95            542     418
  district council        508                     102            725     303
  dwe                    9434                    1622           6349    4351
  government             1195                     322           2175     913
  hesawa                  786                      54            555     373
  kkkt                    425                      62            423     225
  lga                     105                      81            227      93
  other                 15660                    1604           9604    6744
  rwe                     304                     137    

In [38]:
combine$scheme_management[combine$scheme_management %in% c(" ", "", "0", "_", "-")] <- "Other"

In [39]:
# Split DataFrame into training and test sets
test <- combine %>% filter(status_group %in% c('NIL'))
train <- combine %>% filter(!status_group %in% c('NIL'))

To confirm that the factor levels are the same, we can use _levels(table$column)_.

In [40]:
levels(train$scheme_management)

[1] ""                 "Company"          "None"             "Other"           
 [5] "Parastatal"       "Private operator" "SWC"              "Trust"           
 [9] "VWC"              "Water authority"  "Water Board"      "WUA"             
[13] "WUG"

In [41]:
levels(test$scheme_management)

[1] ""                 "Company"          "None"             "Other"           
 [5] "Parastatal"       "Private operator" "SWC"              "Trust"           
 [9] "VWC"              "Water authority"  "Water Board"      "WUA"             
[13] "WUG"

### Select variables that we have picked earlier 

To reiterate, for variables that measure the same thing, we can only keep one such variable to reduce the multi-collinearity effect. In addition, we remove variables that do not have an impact on waterpoint functionality, such as the waterpoint name and data entry recorded date.

In [42]:
train <- train[, which(names(train) %in% c('status_group','amount_tsh','longitude','latitude','gps_height','installer','basin','population','scheme_management','permit','construction_year','payment_type','extraction','management','water_quality','quantity','source','waterpoint_type'))]
test <- test[, which(names(test) %in% c('id','amount_tsh','longitude','latitude','gps_height','installer','basin','population','scheme_management','permit','construction_year','payment_type','extraction','management','water_quality','quantity','source','waterpoint_type'))]

In [43]:
ncol(train)

[1] 17

Now, we have 16 predicor variables and 1 dependent variable.

## Save train and test to CSV files

In [44]:
write.csv(train, "cleanTrain.csv", row.names=FALSE)

In [45]:
write.csv(test, "cleanTest.csv", row.names=FALSE)